In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pandas as pd
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#***TAKE INPUT***

In [4]:
# load 4D data
test_Normal = np.load('/content/drive/My Drive/data/test_Normal_128.npy')
train_virus = np.load('/content/drive/My Drive/data/train_Virus_128.npy')
train_Normal = np.load('/content/drive/My Drive/data/train_Normal_128.npy')
test_virus = np.load('/content/drive/My Drive/data/test_Virus_128.npy')
torch.manual_seed(101)

#***Data Preprocessing***

In [5]:
#Train Data
train_normal = torch.from_numpy(train_Normal)
train_virus = torch.from_numpy(train_virus)
train_normal_label = torch.zeros(len(train_normal))
train_virus_label = torch.ones(len(train_virus))
train_data = torch.cat((train_normal, train_virus), axis=0)
train_label = torch.cat((train_normal_label, train_virus_label), axis=0)
print(train_data.shape)
train_data = torch.reshape(train_data, (2686,1,128,128))


#Test Data
test_normal = test_Normal.astype(np.float32)
test_normal = torch.from_numpy(test_normal)
test_virus = torch.from_numpy(test_virus)
test_normal_label = torch.zeros(len(test_normal))
test_virus_label = torch.ones(len(test_virus))
test_data = torch.cat((test_normal, test_virus), axis=0)
test_label = torch.cat((test_normal_label, test_virus_label), axis=0)
print(test_data.shape)
test_data = torch.reshape(test_data,(382,1,128,128))

torch.Size([2686, 128, 128, 1])
torch.Size([382, 128, 128, 1])


In [6]:
traindata = [train_data[i] for i in range(len(train_data))]
train = torch.stack([d[0] for d in traindata], dim=0)
testdata = [test_data[i] for i in range(len(test_data))]
test = torch.stack([d[0] for d in testdata], dim=0)       

print(len(traindata))
print(test.shape)
print(train.shape)

2686
torch.Size([382, 128, 128])
torch.Size([2686, 128, 128])


In [7]:
train_y = train_label  
test_y = test_label

#***Data shuffling***

In [8]:
train_loader = torch.utils.data.DataLoader(dataset= (train,train_y) , 
                                           shuffle=True)   

test_loader = torch.utils.data.DataLoader(dataset= (test,test_y), 
                                         shuffle=False)

#***Implementing CNN***

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=1, out_channels=16,kernel_size=5, stride=1, padding=2)
        self.cnn_layer_2 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=5, stride=1, padding=2)
        self.cnn_layer_3 = nn.Conv2d(in_channels=32 ,out_channels=64,kernel_size=5, stride=1, padding=2)
        
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(2,2)
        
        self.linear_layer_1 = nn.Linear(64*64*16, 512) 
        self.linear_layer_2 = nn.Linear(512, 256)
        self.linear_layer_3 = nn.Linear(256, 128)
        self.linear_layer_4= nn.Linear(128, 2)   
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.1)
        
        
        
    def forward(self, x):

        x = self.cnn_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x) 
        x = self.sigmoid(x)
        
        x = self.cnn_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x=self.sigmoid(x)

        x = self.cnn_layer_3(x)
        x = self.dropout(x)
        x = self.relu(x)      
        x=self.sigmoid(x)

        x = self.flatten(x)
        
        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)

        
        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_3(x)
        x = self.dropout(x)
        x = self.relu(x)

        x = self.linear_layer_4(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        logits = self.sigmoid(x)
        return x
     

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(128*128)
model.to(device)

NeuralNetwork(
  (cnn_layer_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer_1): Linear(in_features=65536, out_features=512, bias=True)
  (linear_layer_2): Linear(in_features=512, out_features=256, bias=True)
  (linear_layer_3): Linear(in_features=256, out_features=128, bias=True)
  (linear_layer_4): Linear(in_features=128, out_features=2, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.1, inplace=False)
)

#***Implementing Loss function and Optimizer***

In [11]:
learning_rate=0.0005
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
     

#***Train The Model***

In [12]:
def trainModel(model, loss_fn, optimizer):
    model.train()
    batch = 200
    loss = 0
    
    for i in range(train.shape[0]):
      x, y = torch.reshape(train[i],(1,1,128,128)), torch.tensor([train_y[i]], dtype=torch.float)
      label=torch.zeros([1,2], dtype=torch.float32)
      label[0,int(y.item())]=1
      #Compute prediction error
      predicted= model(x)
      loss += criterion(predicted, label)
      
      if i>0 and (i+1)%batch == 0:
          #Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loss = 0
    print('Training Loss:{}'.format(loss.item()))
    print("<====================================================================================>")

#***Test The Model***

In [16]:
   
    size = test.shape[0]
    correct=0
    loss = 0
    iter=240
    total =382
    trainModel(model, criterion, optimizer)
    
    for i in range(iter):
         x, y = torch.reshape(test[i],(1,1,128,128)), torch.tensor([test_y[i]], dtype=torch.float)
         label=torch.zeros([1,2], dtype=torch.float32)
         label[0,int(y.item())]=1
         pred = model(x)  
         _, predicted = torch.max(pred, 1)
          # Total correct predictions
         correct += (predicted == int(y)).sum()
         loss += criterion(pred, label)
      
         loss /= size
         accuracy = 100 * correct.item() / total

         # Print Loss and accuracy
         print('Iteration: {}. || Loss: {}. ||  Accuracy: {}'.format(i+1,loss.item(), accuracy))
         print("<====================================================================================>")
   

Training Loss:59.610618591308594
<====================================================================================>
Iteration: 1. || Loss: 0.0018145213834941387. ||  Accuracy: 0.2617801047120419
<====================================================================================>
Iteration: 2. || Loss: 0.0018192714778706431. ||  Accuracy: 0.5235602094240838
<====================================================================================>
Iteration: 3. || Loss: 0.0018192838178947568. ||  Accuracy: 0.7853403141361257
<====================================================================================>
Iteration: 4. || Loss: 0.0018192840507254004. ||  Accuracy: 1.0471204188481675
<====================================================================================>
Iteration: 5. || Loss: 0.0018192840507254004. ||  Accuracy: 1.3089005235602094
<====================================================================================>
Iteration: 6. || Loss: 0.0018192840507254004. ||  